In [2]:
import time
import faiss
import numpy as np

### faiss チュートリアル

In [59]:
def f(s, diff, dim=10):
    ans = [s]
    for i in range(dim-1):
        s += diff
        ans.append(s)
    return ans

In [76]:
d = 10                           # dimension
nb = 6                   # database size
nq = 1                       # nb of queries
xb = np.asarray([f(0,1), f(0,2), f(10, 1), f(100, 1), f(100,2), f(1000,1)])
print("データ: \n", xb, "\n")
xb = xb.astype('float32')

データ: 
 [[   0    1    2    3    4    5    6    7    8    9]
 [   0    2    4    6    8   10   12   14   16   18]
 [  10   11   12   13   14   15   16   17   18   19]
 [ 100  101  102  103  104  105  106  107  108  109]
 [ 100  102  104  106  108  110  112  114  116  118]
 [1000 1001 1002 1003 1004 1005 1006 1007 1008 1009]] 



In [77]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
6


In [78]:
np.set_printoptions(suppress=True)
k = 6
xq = np.asarray([xb[5] + 0.1]).astype('float32')
print("クエリ: \n", xq, "\n")
D, I = index.search(xq, k)     # actual search
print("距離: \n", np.round(D,3))
print("近傍点のインデックス: \n", I)

クエリ: 
 [[1000.1 1001.1 1002.1 1003.1 1004.1 1005.1 1006.1 1007.1 1008.1 1009.1]] 

距離: 
 [[       0.1  8021076.   8101799.   9802979.   9912276.  10001999. ]]
近傍点のインデックス: 
 [[5 4 3 2 1 0]]


### faiss 計算時間調査

In [65]:
d = 128                # 128次元ベクトル
nb = 100000         # データ数 10万
nq = 1                   # クエリ数 1 
np.random.seed(1234)
xb = np.random.random((nb, d)).astype('float32')  # データ
#xq = np.random.random((nq, d)).astype('float32')  #クエリ
xq = np.asarray([xb[500]]).astype('float32')
print("データの次元: {}".format(xb.shape) )
print("クエリの次元: {}".format(xq.shape))
print("クエリ: \n{}".format(xq))

データの次元: (100000, 128)
クエリの次元: (1, 128)
クエリ: 
[[0.5199999  0.11395431 0.31165713 0.88577837 0.32091683 0.84146696
  0.00297929 0.6017887  0.08570484 0.7624906  0.56640005 0.4257477
  0.8185926  0.8542828  0.11529834 0.78433055 0.86868227 0.35482067
  0.59846205 0.43099362 0.38494176 0.75447476 0.21716215 0.8458164
  0.9966844  0.14915703 0.1918483  0.36647215 0.05130152 0.55463654
  0.82854235 0.9108368  0.41625267 0.53293747 0.8199198  0.65849704
  0.97373575 0.8330749  0.5216754  0.08860879 0.42880335 0.9668112
  0.78999263 0.17166692 0.33181718 0.5395334  0.26246822 0.41571632
  0.5459103  0.08196091 0.96779096 0.34975305 0.4904926  0.3337514
  0.25700626 0.20830499 0.34705496 0.9344718  0.05562545 0.8325989
  0.05277668 0.18765453 0.15717587 0.89648783 0.46726197 0.36992952
  0.5541076  0.49727356 0.6885328  0.9979676  0.6364203  0.31620008
  0.330143   0.29274628 0.4481994  0.31723964 0.13927269 0.4262552
  0.3739465  0.72778475 0.47108287 0.71252257 0.53247404 0.10428359
  0.82486

In [66]:
index = faiss.IndexFlatL2(d)  # インデックス作成
print("インデックス生成済み: {}".format(index.is_trained))
index.add(xb)                          # インデックスにデータを追加
print("インデックス生成済みデータ数: {}".format(index.ntotal))

インデックス生成済み: True
インデックス生成済みデータ数: 100000


In [67]:
np.set_printoptions(suppress=True)
k = index.ntotal                     # 検索する近傍点の数 10万

start = time.time()
D, I = index.search(xq, k)     # 検索
end = time.time()

t = end - start
h = t//3600
m = (t-(3600*h))//60
s = t-(3600*h)-(60*m)

print("距離ベクトルの次元: {}".format(D.shape))
print("近傍点のインデックスの次元: {}".format(I.shape))
print("データ数: {0}, クエリ数: {1}, 検索時間: {2}時間{3}分{4}秒".format(nb, nq, h, m, s))

距離ベクトルの次元: (1, 100000)
近傍点のインデックスの次元: (1, 100000)
データ数: 100000, クエリ数: 1, 検索時間: 0.0時間0.0分0.09517598152160645秒


### faiss 最遠近傍点取得

In [68]:
# 最近傍データ
NN_index = I[0][0]
NN = xb[NN_index]
NN

array([0.5199999 , 0.11395431, 0.31165713, 0.88577837, 0.32091683,
       0.84146696, 0.00297929, 0.6017887 , 0.08570484, 0.7624906 ,
       0.56640005, 0.4257477 , 0.8185926 , 0.8542828 , 0.11529834,
       0.78433055, 0.86868227, 0.35482067, 0.59846205, 0.43099362,
       0.38494176, 0.75447476, 0.21716215, 0.8458164 , 0.9966844 ,
       0.14915703, 0.1918483 , 0.36647215, 0.05130152, 0.55463654,
       0.82854235, 0.9108368 , 0.41625267, 0.53293747, 0.8199198 ,
       0.65849704, 0.97373575, 0.8330749 , 0.5216754 , 0.08860879,
       0.42880335, 0.9668112 , 0.78999263, 0.17166692, 0.33181718,
       0.5395334 , 0.26246822, 0.41571632, 0.5459103 , 0.08196091,
       0.96779096, 0.34975305, 0.4904926 , 0.3337514 , 0.25700626,
       0.20830499, 0.34705496, 0.9344718 , 0.05562545, 0.8325989 ,
       0.05277668, 0.18765453, 0.15717587, 0.89648783, 0.46726197,
       0.36992952, 0.5541076 , 0.49727356, 0.6885328 , 0.9979676 ,
       0.6364203 , 0.31620008, 0.330143  , 0.29274628, 0.44819

In [69]:
# 最遠傍
FW_index = I[0][-1]
FW = xb[FW_index]
FW

array([0.7204853 , 0.86127603, 0.26300618, 0.14949386, 0.7499202 ,
       0.0613309 , 0.9194829 , 0.29523188, 0.18122832, 0.7674253 ,
       0.8793895 , 0.3203156 , 0.93133795, 0.46917826, 0.7554081 ,
       0.6796816 , 0.4467723 , 0.75692207, 0.5655637 , 0.7684299 ,
       0.06474742, 0.65290403, 0.76571506, 0.5063248 , 0.4601451 ,
       0.9203535 , 0.5472907 , 0.33596137, 0.86966664, 0.41865513,
       0.08545654, 0.04614303, 0.31813428, 0.52660197, 0.8999789 ,
       0.14689495, 0.48415148, 0.57409835, 0.7603844 , 0.86042833,
       0.27874926, 0.48184353, 0.23113139, 0.92791504, 0.46754426,
       0.08611915, 0.30608407, 0.44236064, 0.9707559 , 0.9090027 ,
       0.53801507, 0.9878625 , 0.66664237, 0.79058903, 0.21193565,
       0.8211396 , 0.93897736, 0.03074363, 0.4947126 , 0.8054989 ,
       0.82810974, 0.22508715, 0.8242646 , 0.8200726 , 0.62500787,
       0.9322303 , 0.4400758 , 0.09636039, 0.23348917, 0.03907918,
       0.8137552 , 0.4836113 , 0.9465664 , 0.983517  , 0.85946